In [0]:
%run ../config/setup

In [0]:
from pyspark.sql.functions import avg, sum, min, max, col

df_silver = spark.table(full_path_silver)



In [0]:
#Table: hourly
df_hourly = df_silver.groupBy("hour_of_day") \
    .agg(
        avg("consumption_kwh").alias("avg_consumption_kwh"),
        min("voltage").alias("min_voltage"),
        max("voltage").alias("max_voltage"),
        avg("global_intensity").alias("avg_current_intensity")
    ).orderBy("hour_of_day")

df_hourly.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_hourly)